# Comparacion de software legacy con nuevo

El objetivo de este notebook es realizar una comparacion programatica entre los resultados arrojados por ambos softwares.

La mision es que las planillas entregadas por ambos sean identicas, permitiendo iterar de ahora en mas en la nueva version del software. Esta misma.

## Obtencion de los test

Los test fueron guardados con el formato `test-<NAME>.csv`

In [27]:
import os
from pathlib import Path
import zipfile
import pandas as pd    
 
TESTS_DIRECTORY = Path() / "test_data"


def get_all_tests() -> list[str]:
    return [  
        filename.split('-')[-1].split('.')[0]
        for filename in os.listdir(TESTS_DIRECTORY) if 'test' in filename
    ]

all_test = get_all_tests()
print(all_test)

['all_users', 'only_one_user']


## Obtencion de los resultado y armado del diccionario tests-results

Los resultados por el momento son entregados por el software como zip. Dentro del cual estan las planillas csv correspondiente a cada usuario. El formato de los resultados es `result-<TEST-NAME>-<VERSION>.zip`

In [25]:
def get_result_name_for_test(test_name: str) -> str:
    return [filename for filename in os.listdir(TESTS_DIRECTORY) if test_name in filename and not 'test' in filename]


test_and_result = {
    test: get_result_name_for_test(test)
    for test in all_test
}

    

## Comparacion

Los resultados estan dentro del zip, y para eso hay que abrirlos y leerlos. Hay que tener en cuenta que en la version vieja el separador utilizado en los resultado es "punto y coma" `;` y en la nueva version es el default "coma" `,`. 

Para ahcer esto obtive los df de ambos zips como un diccionario por nombre del csv. Y luego realice la comparacion.

In [53]:
def get_all_csv_from_zipfile(zipfile_name: str) -> str:

    with zipfile.ZipFile(TESTS_DIRECTORY/ zipfile_name, 'r') as zip_ref:
        dfs = {}
        for filename in zip_ref.namelist():
            with zip_ref.open(filename) as file: 
                dfs[filename] = pd.read_csv(file, sep=";")
    return dfs


# def make_comparition(test_and_result):
for test, results in test_and_result.items():

    print("test: ", test)
    print("-" * 10)

    zip_result_1 = results[0]
    zip_result_2 = results[1]

    result_df_1 = get_all_csv_from_zipfile(zip_result_1)
    result_df_2 = get_all_csv_from_zipfile(zip_result_2)

    for csv_name, df in result_df_1.items():
        print(f"Checking csv: {csv_name} -- ", end="")
        result_1 = result_df_1[csv_name]
        result_2 = result_df_2[csv_name]
        pd.testing.assert_frame_equal(result_1, result_2)
        print(f'{"✔" if result_1.equals(result_2) else "❌"}')
    print("\n\n")

# make_comparition(test_and_result)

test:  all_users
----------
Checking csv: acattaneo.csv -- ✔
Checking csv: ccomandini.csv -- ✔
Checking csv: dguzman.csv -- ✔
Checking csv: nmanzanarez.csv -- ✔



test:  only_one_user
----------
Checking csv: nmanzanarez.csv -- ✔





### Conclusion

Los dataframe son identicos salvo por la separacion, no se si realmente eso es un problema o no, pero podría configurarse.
